In [1]:
import requests
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager # 0.3.1
import os 
import sys 
from pprint import pprint
sys.path.append('../')

from utils.pdf import read_pdf

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
/home/owen/anaconda3/envs/gen-ai/lib/python3.12/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [8]:


# Chunk the document based on the token limit
def chunk_document(document, chunk_size=3000):
    words = document.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(current_chunk) >= chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(" ".join(current_chunk))  # Add remaining chunk
    
    return chunks

In [9]:
vllm_config = {
    "config_list": [
        {
            "model": "hugging-quants/Llama-3.2-3B-Instruct-Q8_0-GGUF/llama-3.2-3b-instruct-q8_0.gguf", # using LMStudio
            "base_url": "http://localhost:1234/v1",  # Your vLLM URL, with '/v1' added
            "api_key": "lm-studio", # Not needed
        }
    ],
    "cache_seed": None # Turns off caching, useful for testing different models
}

In [10]:
SUMMARIZER_PROMT = \
"""
You are an expert at summarizing text and maintaining context.
Your goal is to extract key points, facts, and topics from the input text.
Generate concise summaries that capture the all of the main points of the text.

Include the following in your summary:
* Source information
* Authors
* Date
* Keywords and concepts

"""


In [11]:
# Create an agent for summarizing
summarizer_agent = AssistantAgent(
    name="Summarizer",
    system_message=SUMMARIZER_PROMT,
    llm_config=vllm_config
)


In [16]:
user_proxy = UserProxyAgent(name="User", llm_config=vllm_config)
agents = [user_proxy, summarizer_agent, ]
group_chat = GroupChat(agents=agents, messages=[], max_round=4)
manager = GroupChatManager(groupchat=group_chat, llm_config=vllm_config)


In [13]:
pdf_text = read_pdf("../data/Sohn_Visual_Prompt_Tuning_for_Generative_Transfer_Learning_CVPR_2023_paper.pdf")

Couldnt get Autogen to work with LMStudio

In [17]:
chunks = chunk_document(pdf_text)
print(f"number of chunks: {len(chunks)}")
current_summary = ""

for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i+1}/{len(chunks)}...")
    
    if i == 0:
        summary_task = f"""
        Summarize the following text concisely:

        {chunk}

        This is the first part of a longer document. Please provide a summary that can be built upon.
        """
    else:
        summary_task = f"""
        Here's the current summary of the document so far:

        {current_summary}

        Now, incorporate the following new chunk of text into the summary, maintaining context and flow:

        {chunk}

        Please provide an updated summary that includes the new information while maintaining the overall context.
        """

    result = user_proxy.initiate_chat(manager, message=summary_task)
    current_summary = result.summary

# Final review and refinement
final_review_task = f"""
Review and refine the following summary of the entire document:

{current_summary}

Please ensure the final summary is well-structured, coherent, and captures the main points of the entire document while maintaining proper context and flow.
"""

final_result = user_proxy.initiate_chat(manager, message=final_review_task)

number of chunks: 3
Processing chunk 1/3...
User (to chat_manager):


        Summarize the following text concisely:

        # Visual Prompt Tuning for Generative Transfer Learning Kihyuk Sohn, Huiwen Chang, Jos´e Lezama, Luisa Polania, Han Zhang, Yuan Hao, Irfan Essa, Lu Jiang Google Research Figure 1. Image synthesis by knowledge transfer. Unlike previous works using GANs as base model and test transfer on relatively narrow visual domains, we transfer knowledge of generative vision transformers [7, 15] to a wide range of visual domains, including natural (e.g., scene, flower), specialized (e.g., satellite, medical), and structured (e.g., road scenes, infograph, sketch) with a few training images. Notably, the prompt tuning significantly improves the prior best FID on two benchmarks ImageNet (85.9!16.3) and Places (71.3!24.2). **Abstract** _Learning generative image models from various domains_ _efficiently needs transferring knowledge from an image syn-_ _thesis model trained on a 

GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.


BadRequestError: Error code: 400 - {'error': '<LM Studio error> vector::_M_range_insert. Error Data: n/a, Additional Data: n/a'}

In [ ]:
def summarize_document(document):
    chunks = chunk_document(document)
    current_summary = ""

    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}...")
        
        if i == 0:
            summary_task = f"""
            Summarize the following text concisely:

            {chunk}

            This is the first part of a longer document. Please provide a summary that can be built upon.
            """
        else:
            summary_task = f"""
            Here's the current summary of the document so far:

            {current_summary}

            Now, incorporate the following new chunk of text into the summary, maintaining context and flow:

            {chunk}

            Please provide an updated summary that includes the new information while maintaining the overall context.
            """

        result = user_proxy.initiate_chat(manager, message=summary_task)
        current_summary = result.summary

    # Final review and refinement
    final_review_task = f"""
    Review and refine the following summary of the entire document:

    {current_summary}

    Please ensure the final summary is well-structured, coherent, and captures the main points of the entire document while maintaining proper context and flow.
    """

    final_result = user_proxy.initiate_chat(manager, message=final_review_task)
    return final_result.summary
